# Neural network to approximate Runge's function

In [1]:
# Imports
import autograd.numpy as np  # We need to use this numpy wrapper to make automatic differentiation work later
from autograd import grad, elementwise_grad
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import (
    PolynomialFeatures,
)
import json

from functions import runge, OLS_parameters, MSE
from functions import ReLU, ReLU_der, sigmoid, sigmoid_der, softmax, softmax_vec, mse_der
from functions import MSE, identity, identity_der, R2
from nn_class import NeuralNetwork

In [3]:
np.random.seed(42)
n = 1000
# data set from project 1
x = np.linspace(-1,1, n).reshape(-1,1)
y = runge(x) + 0.1*np.random.normal(0,1, x.shape)

# Save x and y (flattened) to JSON for easy loading later
xy = {"x": x.ravel().tolist(), "y": y.ravel().tolist()}
with open("data/xy.json", "w") as f:
    json.dump(xy, f)
print("Saved x,y to data/xy.json")

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
y_offset = y_train.mean()

poly = PolynomialFeatures(degree=10)
X_train = poly.fit_transform(x_train)
X_test = poly.fit_transform(x_test)
# scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

# OLS
beta = OLS_parameters(X_train_s, y_train)
y_test_tilde = X_test_s @ beta + y_offset
y_train_tilde = X_train_s @ beta + y_offset

targets = y_test
print('OLS mse:', MSE(targets, y_test_tilde))
print('OLS R2:', R2(targets, y_test_tilde))

mse_OLS = MSE(targets, y_test_tilde)
R2_OLS = R2(targets, y_test_tilde)

metrics = {"mse": float(mse_OLS), "r2": float(R2_OLS)}
with open("data/ols_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

print("Saved OLS metrics to ols_metrics.json")

# Smooth Runge curve and OLS approximation over a fine grid
xx = np.linspace(-1, 1, 400).reshape(-1, 1)
yy = runge(xx)

# Create polynomial features for the grid, scale and predict with OLS beta
XX = poly.transform(xx)
XX_s = scaler.transform(XX)
y_ols = XX_s @ beta + y_offset

plot_data = {"xx": xx.ravel().tolist(), "yy": yy.ravel().tolist(), "y_ols": y_ols.ravel().tolist()}
with open("data/xx_yy_yols.json", "w") as f:
    json.dump(plot_data, f, indent=2)
print("Saved xx, yy, y_ols to data/xx_yy_yols.json")

Saved x,y to data/xy.json
OLS mse: 0.010975094308561558
OLS R2: 0.8471780118227936
Saved OLS metrics to ols_metrics.json
Saved xx, yy, y_ols to data/xx_yy_yols.json


In [9]:
# One layer, 50 neurons, sigmoid activation function, plain gradient descent, no optimizations
layer_output_sizes = [50, 1]
activation_funcs = [sigmoid, identity]
activation_ders = [sigmoid_der, identity_der]

scaler.fit(x_train)
x_train_s = scaler.transform(x_train)
x_test_s = scaler.transform(x_test)
inputs = x_train_s
targets = y_train

NN = NeuralNetwork(
    x_train_s,
    targets,
    layer_output_sizes,
    activation_funcs,
    activation_ders,
    MSE,
    mse_der,
)

NN.train_network_plain_gd(max_iter=1000000000)

targets = y_test

predictions = NN.predict(x_test_s)
print('NN mse:', MSE(predictions, targets))
print('NN R2:', R2(predictions, targets))

mse_NN_50_sigmoid_plain_no = MSE(predictions, targets)
R2_NN_50_sigmoid_plain_no = R2(predictions, targets)

metrics = {"mse": float(mse_NN_50_sigmoid_plain_no), "r2": float(R2_NN_50_sigmoid_plain_no)}
with open("data/nn_50_sigmoid_plain_no_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

# Save best model predictions for plotting
x_plot = np.linspace(-1, 1, 400).reshape(-1, 1)
y_pred_plot = NN.predict(x_plot).reshape(-1)

plot_data = {"x_plot": x_plot.tolist(), "y_pred_plot": y_pred_plot.tolist()}
with open("data/x_plot_predictions_50_sigmoid_plain_no_metrics.json", "w") as f:
    json.dump(plot_data, f, indent=2)

Early stopping at iteration 723972, cost change 1.00e-10 <= 1e-10
NN mse: 0.009559322081448109
NN R2: 0.8835234659607542


In [ ]:
# One layer, 100 neurons, sigmoid activation function, plain gradient descent, no optimizations
layer_output_sizes = [100, 1]
activation_funcs = [sigmoid, identity]
activation_ders = [sigmoid_der, identity_der]

scaler.fit(x_train)
x_train_s = scaler.transform(x_train)
x_test_s = scaler.transform(x_test)
inputs = x_train_s
targets = y_train

NN = NeuralNetwork(
    x_train_s,
    targets,
    layer_output_sizes,
    activation_funcs,
    activation_ders,
    MSE,
    mse_der,
)

NN.train_network_plain_gd(max_iter=1000000000)

targets = y_test

predictions = NN.predict(x_test_s)
print('NN mse:', MSE(predictions, targets))
print('NN R2:', R2(predictions, targets))

mse_NN_100_sigmoid_plain_no = MSE(predictions, targets)
R2_NN_100_sigmoid_plain_no = R2(predictions, targets)

metrics = {"mse": float(mse_NN_100_sigmoid_plain_no), "r2": float(R2_NN_100_sigmoid_plain_no)}
with open("data/nn_100_sigmoid_plain_no_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

In [5]:
# 2 layer, 50 neurons, sigmoid activation function, plain gradient descent, no optimizations
layer_output_sizes = [50, 50, 1]
activation_funcs = [sigmoid, sigmoid, identity]
activation_ders = [sigmoid_der, sigmoid_der, identity_der]

scaler.fit(x_train)
x_train_s = scaler.transform(x_train)
x_test_s = scaler.transform(x_test)
inputs = x_train_s
targets = y_train

NN = NeuralNetwork(
    x_train_s,
    targets,
    layer_output_sizes,
    activation_funcs,
    activation_ders,
    MSE,
    mse_der,
)

NN.train_network_plain_gd(max_iter=1000000000)

targets = y_test

predictions = NN.predict(x_test_s)
print('NN mse:', MSE(predictions, targets))
print('NN R2:', R2(predictions, targets))

mse_NN_50_50_sigmoid_plain_no = MSE(predictions, targets)
R2_NN_50_50_sigmoid_plain_no = R2(predictions, targets)

metrics = {"mse": float(mse_NN_50_50_sigmoid_plain_no), "r2": float(R2_NN_50_50_sigmoid_plain_no)}
with open("data/nn_50_50_sigmoid_plain_no_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

# Save best model predictions for plotting
x_plot = np.linspace(-1, 1, 400).reshape(-1, 1)
y_pred_plot = NN.predict(x_plot).reshape(-1)

plot_data = {"x_plot": x_plot.tolist(), "y_pred_plot": y_pred_plot.tolist()}
with open("data/x_plot_predictions_50_50_sigmoid_plain_no_metrics.json", "w") as f:
    json.dump(plot_data, f, indent=2)

Early stopping at iteration 482466, cost change 1.00e-10 <= 1e-10
NN mse: 0.009425809235790925
NN R2: 0.8851446744512326


In [8]:
x_plot = np.linspace(-1, 1, 1000).reshape(-1, 1)
y_pred_plot = NN.predict(x_plot).reshape(-1)

plot_data = {"x_plot": x_plot.tolist(), "y_pred_plot": y_pred_plot.tolist()}
with open("data/x_plot_predictions_50_50_sigmoid_plain_no_metrics.json", "w") as f:
    json.dump(plot_data, f, indent=2)

In [ ]:
# 2 layer, 100 neurons, sigmoid activation function, plain gradient descent, no optimizations
layer_output_sizes = [100, 100, 1]
activation_funcs = [sigmoid, sigmoid, identity]
activation_ders = [sigmoid_der, sigmoid_der, identity_der]

scaler.fit(x_train)
x_train_s = scaler.transform(x_train)
x_test_s = scaler.transform(x_test)
inputs = x_train_s
targets = y_train

NN = NeuralNetwork(
    x_train_s,
    targets,
    layer_output_sizes,
    activation_funcs,
    activation_ders,
    MSE,
    mse_der,
)

NN.train_network_plain_gd(max_iter=1000000000)

targets = y_test

predictions = NN.predict(x_test_s)
print('NN mse:', MSE(predictions, targets))
print('NN R2:', R2(predictions, targets))

mse_NN_100_100_sigmoid_plain_no = MSE(predictions, targets)
R2_NN_100_100_sigmoid_plain_no = R2(predictions, targets)

metrics = {"mse": float(mse_NN_100_100_sigmoid_plain_no), "r2": float(R2_NN_100_100_sigmoid_plain_no)}
with open("data/nn_100_100_sigmoid_plain_no_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)